# US Census - Extraction

### Import Libraries

In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Parameters

In [23]:
URL = "https://data.census.gov/table?q=B25040&g=160XX00US0643000&tid=ACSDT1Y2021.B25040"
API_METADATA = "https://data.census.gov/api/search/metadata/table?id=ACSDT1Y2021.B25040&g=160XX00US0643000"
API_TABLE = "https://data.census.gov/api/access/data/table?id=ACSDT1Y2021.B25040&g=160XX00US0643000"

In [24]:
headers =  {
    "authority": "data.census.gov",
    "method": "GET",
    "path": "/table?q=B25040&g=160XX00US0643000&tid=ACSDT1Y2021.B25040",
    "scheme": "https",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Cache-Control": "max-age=0",
    "Sec-Ch-Ua": "\"Not.A/Brand\";v=\"8\", \"Chromium\";v=\"114\", \"Google Chrome\";v=\"114\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

### Tests

In [29]:
r = requests.get(API_METADATA, headers=headers)
metadata_content = r.json()
metadata_content['response']

{'metadataContent': {'measures': [{'item_classification': {'id': 'ESTIMATE',
     'label': 'Estimate'},
    'attributes': [{'displayType': {'displayTypeLabel': 'SHOW_BOTH_COLS',
       'displayType': 'SHOW_BOTH_COLS'},
      'attribute_type': {'type': 'MARGIN_OF_ERROR',
       'typeLabel': 'MARGIN_OF_ERROR'},
      'attributes': [{'item_data_behavior_type': {'description': 'DIRECT',
         'id': 'DIRECT'},
        'displayType': {'displayTypeLabel': 'SHOW_BOTH_COLS',
         'displayType': 'SHOW_BOTH_COLS'},
        'attribute_type': {'type': 'ANNOTATION', 'typeLabel': 'ANNOTATION'},
        'id': 'B25040_001MA',
        'label': 'Annotation of Margin of Error!!Total:',
        'unit_type': {'unit_type_label': 'NAN', 'id': 'NAN'}}],
      'id': 'B25040_001M',
      'label': 'Margin of Error!!Total:',
      'unit_type': {'unit_type_label': 'MOE', 'id': 'MOE'}},
     {'item_data_behavior_type': {'description': 'DIRECT', 'id': 'DIRECT'},
      'displayType': {'displayTypeLabel': 'SHOW_

In [51]:
metadata_content['response']['metadataContent']['dataset']

{'component': 'Detailed Tables',
 'provider': 'ACS',
 'vintage': '2021',
 'dataCategory': 'AGGREGATE',
 'name': 'ACS 1-Year Estimates Detailed Tables',
 'program': 'American Community Survey',
 'subProgram': '1-Year Estimates',
 'dataset': 'ACSDT1Y2021',
 'programShortLabel': 'ACS'}

In [58]:
metadata_content['response']['metadataContent']['dimensions'][1]

{'globalDimension': False,
 'dimension_type': {'description': 'MEASURE', 'id': 'MEASURE'},
 'id': 'ESTIMATE',
 'categories': [{'orderNumber': '1',
   'id': 'COLUMN1',
   'label': 'Estimate',
   'item_mapping': ['B25040_001E',
    'B25040_002E',
    'B25040_003E',
    'B25040_004E',
    'B25040_005E',
    'B25040_006E',
    'B25040_007E',
    'B25040_008E',
    'B25040_009E',
    'B25040_010E']},
  {'orderNumber': '2',
   'id': 'COLUMN2',
   'label': 'Margin of Error',
   'item_mapping': ['B25040_001M',
    'B25040_002M',
    'B25040_003M',
    'B25040_004M',
    'B25040_005M',
    'B25040_006M',
    'B25040_007M',
    'B25040_008M',
    'B25040_009M',
    'B25040_010M']}],
 'axis': 'H',
 'order_number': 2}

In [57]:
metadata_content['response']['metadataContent']['dimensions'][2]['categories']

[{'orderNumber': '1',
  'id': 'LINE1',
  'label': 'Total:',
  'item_mapping': ['B25040_001E', 'B25040_001M']},
 {'parent': 'LINE1',
  'orderNumber': '2',
  'id': 'LINE2',
  'label': 'Utility gas',
  'item_mapping': ['B25040_002E', 'B25040_002M']},
 {'parent': 'LINE1',
  'orderNumber': '3',
  'id': 'LINE3',
  'label': 'Bottled, tank, or LP gas',
  'item_mapping': ['B25040_003E', 'B25040_003M']},
 {'parent': 'LINE1',
  'orderNumber': '4',
  'id': 'LINE4',
  'label': 'Electricity',
  'item_mapping': ['B25040_004E', 'B25040_004M']},
 {'parent': 'LINE1',
  'orderNumber': '5',
  'id': 'LINE5',
  'label': 'Fuel oil, kerosene, etc.',
  'item_mapping': ['B25040_005E', 'B25040_005M']},
 {'parent': 'LINE1',
  'orderNumber': '6',
  'id': 'LINE6',
  'label': 'Coal or coke',
  'item_mapping': ['B25040_006E', 'B25040_006M']},
 {'parent': 'LINE1',
  'orderNumber': '7',
  'id': 'LINE7',
  'label': 'Wood',
  'item_mapping': ['B25040_007E', 'B25040_007M']},
 {'parent': 'LINE1',
  'orderNumber': '8',
  'i

In [33]:
r = requests.get(API_TABLE, headers=headers)
table_content = r.json()['response']

In [40]:
labels = table_content['data'][0]
data = table_content['data'][1]

df = pd.DataFrame.from_dict({"labels": labels, "data": data})
df

,labels,data
0,B25040_007M,96
1,B25040_001E,172599
2,B25040_009M,186
3,B25040_003E,1688
4,B25040_005E,106
5,B25040_007E,56
6,B25040_009E,207
7,B25040_001MA,None
8,B25040_002MA,None
9,B25040_003MA,None
